In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
import  analysis.lib.purification.purify_pq as ppq

%matplotlib inline

thetas =['pi/4','pi/5','pi/6','pi/8']

outcome_bins = np.arange(50,540,50)

In [ ]:
def calculate_ebit_rate(outcome_bins,apply_ROC = True):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations('bin_w2',outcome_bins,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
    
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    total_counts_p = psi_p_XX[0]+psi_p_YY[0]+psi_p_ZZ[0]
    total_counts_m = psi_m_XX[0]+psi_m_YY[0]+psi_m_ZZ[0]
    
    total_time = Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    rate_p = np.array(total_counts_p)/total_time
    rate_m = np.array(total_counts_m)/total_time
    rate_p_u = np.sqrt(np.array(total_counts_p))/total_time
    rate_m_u = np.sqrt(np.array(total_counts_m))/total_time
    
    ebit_rate_p = np.sum(rate_p*ebits_p)
    ebit_rate_p_u = np.sqrt(np.sum(rate_p**2*ebits_p_u**2)+np.sum(rate_p_u**2*ebits_p**2))
    ebit_rate_m = np.sum(rate_m*ebits_m)
    ebit_rate_m_u = np.sqrt(np.sum(rate_m**2*ebits_m_u**2)+np.sum(rate_m_u**2*ebits_m**2))
    
    
    avg_fidelity_p = np.sum(psi_p_F*total_counts_p)/np.sum(total_counts_p)
    avg_fidelity_p_u = np.sqrt(np.sum(psi_p_F_u**2*total_counts_p**2))/np.sum(total_counts_p)
    avg_fidelity_m = np.sum(psi_m_F*total_counts_p)/np.sum(total_counts_m)
    avg_fidelity_m_u = np.sqrt(np.sum(psi_m_F_u**2*total_counts_p**2))/np.sum(total_counts_m)
    
    avg_ebits_p = np.sum(ebits_p*total_counts_p)/np.sum(total_counts_p)
    avg_ebits_p_u = np.sqrt(np.sum(ebits_p_u**2*total_counts_p**2))/np.sum(total_counts_p)
    avg_ebits_m = np.sum(ebits_m*total_counts_m)/np.sum(total_counts_p)
    avg_ebits_m_u = np.sqrt(np.sum(ebits_m_u**2*total_counts_m**2))/np.sum(total_counts_p)
    
#     fig  = plt.figure()
#     ax = plt.subplot()
#     ax.set_xlabel('Superposn ms=0 frac.')
#     ax.set_ylabel('Rate')
#     plt.plot(outcome_bins,rate_p)
#     plt.plot(outcome_bins,rate_m)
#     plt.show()
#     plt.close('all')

    
    rate = rate_p + rate_m
    rate_u = np.sqrt(rate_p_u**2 + rate_m_u**2)
    ebits = (ebits_p + ebits_m)/2
    ebits_u = np.sqrt(ebits_p_u**2 + ebits_m_u**2)/2    
    
    ebit_rate = np.sum(rate*ebits)
    ebit_rate_u = np.sqrt(np.sum(rate_u**2*ebits**2)+np.sum(rate**2*ebits_u**2))
    
    rate_p = np.sum(rate_p)
    rate_m = np.sum(rate_m)
    
    return [rate_p,rate_m,
            ebit_rate_p,
            ebit_rate_p_u,
            ebit_rate_m,
            ebit_rate_m_u,
            avg_fidelity_p,
            avg_fidelity_p_u,
            avg_fidelity_m,
            avg_fidelity_m_u,
            avg_ebits_p,
            avg_ebits_p_u,
            avg_ebits_m,
            avg_ebits_m_u,
            rate,
            rate_u,
            ebits,
            ebits_u,
            ebit_rate,
            ebit_rate_u]

In [ ]:
vals = []

for theta in thetas:
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    Purify_XX.get_tstamps_and_offsets(contains = 'XX')
    Purify_YY.get_tstamps_and_offsets(contains = 'YY')
    Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')

#     ### load the data
    Purify_XX.load_raw_data()
    Purify_XX.correct_pq_times()
    Purify_YY.load_raw_data()
    Purify_YY.correct_pq_times()
    Purify_ZZ.load_raw_data()
    Purify_ZZ.correct_pq_times()
    
    vals.append(calculate_ebit_rate(outcome_bins = outcome_bins))
   

## Calculations for other bounds

In [ ]:
TPQI_vis = 0.725
TPQI_vis_u = 0.03

estimatedFirstClickProb = 0.00119647534846  # Multiply by two since at 50% superpos.
estimatedFirstClickProb_u = 1.35775223144e-05
tailLT3 = 10.
tailLT4 = 5.
firstClickProbLT3 = (tailLT3/(tailLT4+tailLT3))*estimatedFirstClickProb
firstClickProbLT4 = (tailLT4/(tailLT4+tailLT3))*estimatedFirstClickProb

firstClickProbLT3_u = (tailLT3/(tailLT4+tailLT3)) * estimatedFirstClickProb_u
firstClickProbLT4_u = (tailLT4/(tailLT4+tailLT3)) * estimatedFirstClickProb_u

pBK = 0.5*firstClickProbLT3*firstClickProbLT4
estimatedRateBK = pBK / (7e-6)

estimatedRateBK_u = ( 0.5 / (7e-6))*np.sqrt(firstClickProbLT3**2*firstClickProbLT4_u**2+firstClickProbLT4**2*firstClickProbLT3_u**2)
estimatedEbitsBK= np.log2(1+TPQI_vis)
estimatedEbitsBK_u = TPQI_vis_u/((1+TPQI_vis)*np.log(2))
# print pBK
# print estimatedRateBK
# print estimatedEbitsBK

estimatedEbitRateBK = estimatedRateBK*estimatedEbitsBK
estimatedEbitRateBK_u = np.sqrt(estimatedEbitsBK_u**2 * estimatedRateBK**2 + estimatedEbitsBK**2 * estimatedRateBK_u**2)

In [ ]:
pur_model_populations = np.array([0.00615583, 0.013815, 0.0244717, 0.0380602, 0.0544967, 0.0736799, \
0.0954915, 0.119797, 0.146447, 0.175276, 0.206107, 0.238751, \
0.273005, 0.308658, 0.345492, 0.383277, 0.421783, 0.46077, 0.5, \
0.53923, 0.578217, 0.616723, 0.654508, 0.691342, 0.726995])

pur_model_ebit_rate = np.array([0.0000899473, 0.00074822, 0.00289022, 0.00725502, 0.0143862, \
0.0243851, 0.0367586, 0.0503864, 0.0636327, 0.0748553, 0.0844905, \
0.0929262, 0.0987514, 0.100784, 0.0984393, 0.0916397, 0.0807205, \
0.0665894, 0.0505063, 0.033916, 0.0187401, 0.00690227, 0.000385282, \
0, 0])

In [ ]:
pur_model_pops = np.array([[0.492, 0.398, 0.276, 0.176]])
pur_model_rate = np.array([[0.252012, 0.271425, 0.300922, 0.327996, 0.345109, 0.351162, \
0.349004, 0.342236, 0.333471, 0.324106], [0.201925, 0.207501, \
0.21905, 0.23063, 0.238094, 0.240409, 0.23872, 0.234739, 0.229759, \
0.224477], [0.122585, 0.119683, 0.119414, 0.120792, 0.12216, \
0.122505, 0.12175, 0.120279, 0.118475, 0.116563], [0.0592885, \
0.0559168, 0.0535633, 0.052619, 0.0524814, 0.0524632, 0.0522616, \
0.0518737, 0.051383, 0.050853]])
pur_model_ebits = np.array([[0.170048, 0.0405187, 0, 0, 0, 0, 0, 0, 0, 0], [0.275868, 0.134293, \
0.0197372, 0, 0, 0, 0, 0, 0, 0], [0.396828, 0.266586, 0.124535, \
0.0302415, 0.000066998, 0, 0, 0, 0, 0], [0.480924, 0.371945, \
0.234841, 0.122756, 0.0605305, 0.0334802, 0.0205731, 0.011426, \
0.00297588, 0]])

In [ ]:
print np.round(ebits,3)
print np.round(ebits_u,3)

print rate_u/rate

In [ ]:
total_ebit_rate_u

In [ ]:
thetas = [np.pi/4,np.pi/5,np.pi/6,np.pi/8]

data = lambda x : np.array([vals[i][x] for i in range(len(vals))])
rate_p = data(0)
rate_m = data(1)
ebit_rate_p = data(2)
ebit_rate_p_u = data(3)
ebit_rate_m = data(4)
ebit_rate_m_u = data(5)
avg_fidelity_p = data(6)
avg_fidelity_p_u = data(7)
avg_fidelity_m = data(8)
avg_fidelity_m_u = data(9)
avg_ebits_p = data(10)
avg_ebits_p_u = data(11)
avg_ebits_m = data(12)
avg_ebits_m_u =data(13)
rate = data(14)
rate_u = data(15)
ebits = data(16)
ebits_u = data(17)
total_ebit_rate = data(18)
total_ebit_rate_u = data(19)

total_ebit_rate_u_lower = [np.min([total_ebit_rate_u[i],total_ebit_rate[i]])\
                           for i in range(np.shape(total_ebit_rate_u)[0])]

total_ebit_rate_u_asym = [total_ebit_rate_u_lower,total_ebit_rate_u]
super_posn_frac = np.sin(thetas)**2

# total_ebit_rate = ebit_rate_p + ebit_rate_m
# total_ebit_rate_u = np.sqrt(ebit_rate_p_u**2 + ebit_rate_m_u**2)


real_theta_vals = np.flipud((0.5-np.array([0.153355,0.2525,0.2827,0.323754,0.362197,0.3865]))
)*np.pi
real_pop_vals = np.sin(real_theta_vals)**2
#needs reversing:
real_pop_vals = real_pop_vals[::-1]


save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*89
fig = plt.figure(figsize =(figure_width,figure_width/golden_ratio))
ax = plt.subplot(111)

lw = 1
fontsize = 10
markersize = 4

# ax.set_xlabel(r'Population in ms=0')
ax.set_xlabel(r'sin$^2 \theta$')
ax.set_ylabel('Ebit rate (Hz)')
#ax.set_yscale("log", nonposy='clip')

plt.plot(pur_model_populations,pur_model_ebit_rate,linestyle='--',color='#3594F2',lw=lw)
plt.errorbar(real_pop_vals[1:-1],total_ebit_rate,total_ebit_rate_u_asym,marker='o',ms=markersize,mew =0.5,capsize=3, 
                     elinewidth=lw,ls='none',label = 'Pur',color='#3594F2',zorder= 10)
# plt.plot([0, 0.6], [estimatedEbitRateBK, estimatedEbitRateBK], label = 'BK')
plt.plot([0, 0.6],np.ones(2)* estimatedEbitRateBK, '#F29335', color='#F29335',label = 'BK',lw=lw)
plt.fill_between([0, 0.6],np.ones(2)*(estimatedEbitRateBK-estimatedEbitRateBK_u),np.ones(2)*( estimatedEbitRateBK+estimatedEbitRateBK_u),
    alpha=0.25, edgecolor='#F29335', facecolor='#F29335',linewidth=0.0)

plt.plot([0, 0.6],np.ones(2)* estimatedRateBK, '--', color='k',label = 'BK perfect',lw=lw)
# plt.fill_between([0, 0.6],np.ones(2)*(estimatedRateBK-estimatedRateBK_u),np.ones(2)*( estimatedRateBK+estimatedRateBK_u),
#     alpha=0.5, edgecolor='#3F7F4C', facecolor='#7EFF99')

plt.xlim([0.1,0.55])
plt.ylim([0.0,1.05*plt.ylim()[1]])
# plt.legend( numpoints = 1,loc=3)

###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)



plt.savefig(os.path.join(save_folder,'ebit_rate.png'),
                    format='png',bbox_inches = 'tight',pad_inches=0.3)
plt.savefig(os.path.join(save_folder,'ebit_rate.pdf'),
                    format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')

# fig  = plt.figure()
# ax = plt.subplot()
# ax.set_xlabel('Superposn ms=0 frac.')
# ax.set_ylabel('Raw rate (Hz)')
# plt.plot(super_posn_frac,rate_m,label = '-')
# plt.plot(super_posn_frac,rate_p,label = '+')
# plt.xlim([0.1,0.55])
# plt.legend()
# plt.show()
# plt.close('all')

# fig  = plt.figure()
# ax = plt.subplot()
# ax.set_xlabel('Superposn ms=0 frac.')
# ax.set_ylabel('Avg fidelity')
# plt.errorbar(super_posn_frac,avg_fidelity_m,avg_fidelity_m_u,label = '-')
# plt.errorbar(super_posn_frac,avg_fidelity_p,avg_fidelity_p_u,label = '+')
# plt.xlim([0.1,0.55])
# plt.legend()
# plt.show()
# plt.close('all')


# fig  = plt.figure()
# ax = plt.subplot()
# ax.set_xlabel('Superposn ms=0 frac.')
# ax.set_ylabel('Avg ebits')
# plt.errorbar(super_posn_frac,avg_ebits_m,avg_ebits_m_u,label = '-')
# plt.errorbar(super_posn_frac,avg_ebits_p,avg_ebits_p_u,label = '+')
# plt.xlim([0.1,0.55])
# plt.legend()
# plt.show()
# plt.close('all')

In [ ]:
def plot_values_bars(x_list,y_list,y_u_list,color_list):
    
    for x,y,y_u,color in zip(x_list,y_list,y_u_list,color_list):
        plt.errorbar(x-25,y,y_u,fmt = 'o',color = color,ms=markersize,mew=lw/2.,zorder=3) #formatting still needs to be done
        ### we want to include bars to signify integration here
        barwidth = (x[1]-x[0])/2.

        alpha = 0.04
        plt.bar(x-barwidth-12.5,y,width = barwidth,alpha = alpha,color = color,zorder=0)

golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*89
fig = plt.figure(figsize =(2*figure_width,figure_width/golden_ratio))

lw = 1
fontsize = 10
markersize = 4

ax = plt.subplot(121)

ax.set_ylabel('Rate (Hz)')
ax.set_xlabel('Entangling attempts (binned)',size = fontsize)

maxbin = 5;

x_list = [(outcome_bins[:maxbin])]*5

colors = ['#3594F2','#f29335','#35f293','#f23594']
for i in range(np.shape(pur_model_rate)[0]):
    plt.plot(outcome_bins[:maxbin]-25,pur_model_rate[i][:maxbin],color=colors[i],linestyle='--',lw=lw)
plot_values_bars(x_list,rate[:,:maxbin],rate_u[:,:maxbin],colors)

plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

ax = plt.subplot(122)

# ax.set_xlabel(r'Population in ms=0')
ax.set_ylabel('$E_N$')
ax.set_xlabel('Entangling attempts (binned)',size = fontsize)

plot_values_bars(x_list,ebits[:,:maxbin],ebits_u[:,:maxbin],colors)
for i in range(np.shape(pur_model_rate)[0]):
    plt.plot(outcome_bins[:maxbin]-25,pur_model_ebits[i][:maxbin],color=colors[i],linestyle='--',lw=lw)

plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
###some plot cosmetics
ax.tick_params(labelsize = fontsize,width =lw)
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(lw)
ax.yaxis.label.set_size(fontsize)
ax.xaxis.label.set_size(fontsize)

legend_list = [r'$\pi/4$',r'$\pi/5$', r'$\pi/6$', r'$\pi/8$']
ax.legend(legend_list, borderaxespad=0.,fontsize =fontsize-1,frameon=False,ncol=2,numpoints=1,
                                  handlelength=0,handletextpad = 0.4,columnspacing = 0.8)
plt.tight_layout()
# plt.savefig(os.path.join(save_folder,'binned_ebits_rate.png'),
#                     format='png',bbox_inches = 'tight',pad_inches=0.3)
# plt.savefig(os.path.join(save_folder,'binned_ebits_rate.pdf'),
#                     format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()
plt.close('all')


# How many phase correction repetitions do we apply?

In [ ]:
lt3_attempts_second = Purify_ZZ.lt3_dict['attempts_second']
lt3_feedback_attempts = Purify_ZZ.lt3_dict['Phase_correction_repetitions']


In [ ]:
phase_reps = np.array([])
max_attempts = 50
for atts,feedback in zip(lt3_attempts_second,lt3_feedback_attempts):
    bin_fltr = lt3_attempts_second < max_attempts+1
    phase_reps = np.concatenate((phase_reps,lt3_feedback_attempts[bin_fltr]))
    
n,bins = np.histogram(phase_reps,bins=60,range=[0,60])

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.plot(bins[:-1],n,'o')
plt.xlabel('Phase feedback repetitions')
plt.ylabel('Occurences')
plt.show()
plt.close('all')

### get the average number of phase reps:
avg_reps = np.sum(n*bins[:-1])/np.sum(n)
print 'average number of phase feedback reps', avg_reps
### average phase feedback duration:
print 'average elapsed feedback time in ms: ',4*avg_reps*Purify_ZZ.lt3_dict['data_attrs'][0]['dynamic_phase_tau']*1e3